# Boston 주택 가격 예측 모델
### 주택의 여러 요건과 가격 정보를 바탕으로 주택 가격 예측

In [2]:
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
# keras.datasets 안에 boston_housing 데이터셋을 로드합니다.
from keras.datasets import boston_housing

(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

# 데이터셋의 크기를 확인합니다.
print(len(train_data))
print(len(test_data))

57026/57026 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
404
102


## 데이터 표준화

In [4]:

# 먼저 입력 데이터의 각 특성의 평균 뺌
mean = train_data.mean(axis = 0)
train_data = train_data - mean

# 표준 편차로 나눔
std = train_data.std(axis = 0)
train_data = train_data / std

# 훈련, 검증 데이터 분리

x_train, x_val, y_train, y_val = train_test_split(train_data, train_labels,
                                                  test_size=0.3,
                                                  random_state=123)

test_data = test_data - mean
test_data = test_data / std


print('train 훈련 데이터 : ', x_train.shape)
print('train 검증 데이터 : ', x_val.shape)
print('test 데이터 : ', test_data.shape)


train 훈련 데이터 :  (282, 13)
train 검증 데이터 :  (122, 13)
test 데이터 :  (102, 13)


### lose : mse(평균 제곱 오차)
### optimizer : Adam
### 지표 : mae(평균 절대 오차)

In [23]:
from tensorflow.keras import models, layers, optimizers

# 모델 구성
model = models.Sequential()
model.add(layers.Input(shape=(train_data.shape[1], )))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1))

model.compile(loss='mse',
              optimizer= 'adam',
              metrics=['mae'])

model.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                     │ (None, 32)                  │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,009 (43.00 KB)

 Trainable params: 11,009 (43.00 KB)

 Non-trainable params: 0 (0.00 B)

## ModelCheckPoint 및 early stopping 적용

In [24]:
from keras import callbacks

check_point_cb = callbacks.ModelCheckpoint('best_model.keras', save_best_only=True)
early_stopping_cb = callbacks.EarlyStopping(patience=3, monitor='val_loss',
                                  restore_best_weights=True)

history = model.fit(x_train, y_train,
                    epochs=300,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=[check_point_cb, early_stopping_cb])

Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 566.5680 - mae: 21.9384 - val_loss: 605.6221 - val_mae: 22.8084
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 562.3679 - mae: 21.8375 - val_loss: 601.2731 - val_mae: 22.7103
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 558.1133 - mae: 21.7341 - val_loss: 596.8434 - val_mae: 22.6095
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 553.7869 - mae: 21.6275 - val_loss: 592.2997 - val_mae: 22.5052
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 549.3607 - mae: 21.5172 - val_loss: 587.6144 - val_mae: 22.3965
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 544.7995 - mae: 21.4020 - val_loss: 582.7524 - val_mae: 22.2829
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 540.0539 - mae: 21.2812 - val_loss: 577.6878 - val_mae: 22.1635
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 535.0917 - mae: 21.1539 - val_loss: 572.4011 - val_mae: 22.0378
Epoch 9/300
1/1 ━━━━━━━━

## 모델 평가 결과 : MAE >> 3.2

In [25]:
model.evaluate(test_data, test_labels, verbose=1)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 18.3293 - mae: 3.0128 


[25.600671768188477, 3.230532169342041]